In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Tham số
csv_file = 'data.csv'
num_sensors = 5
input_channels = num_sensors
num_classes = 36
batch_size = 32
num_epochs = 10
sequence_length = 10

In [2]:
# Định nghĩa mô hình CNN 1D
class CNN1D(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.conv4 = nn.Conv1d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv5 = nn.Conv1d(in_channels=512, out_channels=1024, kernel_size=3, padding=1)
        self.batch_norm1 = nn.BatchNorm1d(128)
        self.batch_norm2 = nn.BatchNorm1d(256)
        self.batch_norm3 = nn.BatchNorm1d(512)
        self.batch_norm4 = nn.BatchNorm1d(512)
        self.batch_norm5 = nn.BatchNorm1d(1024)
        self.dropout = nn.Dropout(0.2)
        self.padding = nn.ConstantPad1d((0, 1), value=0)
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=1 )
        self.fc = nn.Linear(1024*sequence_length, num_classes)

    def forward(self, x):
        x = torch.relu(self.batch_norm1(self.conv1(x)))
        print(f'conv1 {x.shape}')
        x = torch.relu(self.batch_norm2(self.conv2(x)))
        print(f'conv2 {x.shape}')
        x = torch.relu(self.batch_norm3(self.conv3(x)))
        print(f'conv3 {x.shape}')
        x = torch.relu(self.batch_norm4(self.conv4(x)))
        print(f'conv4 {x.shape}')
        x = torch.relu(self.batch_norm5(self.conv5(x)))
        print(f'conv5 {x.shape}')
        x = self.padding(x)
        print(f'padding {x.shape}')
        x = self.maxpool(x)
        print(f'maxpol {x.shape}')
        x = self.dropout(x)
        print(f'dropout {x.shape}')
        x = x.view(x.size(0), -1)  
        print(f'x.view {x.shape}')
        x = self.fc(x)
        return x
# Khởi tạo mô hình
input_size = num_sensors
model = CNN1D(input_size=input_size, num_classes=num_classes)

In [3]:
def convert_to_float32(series):
    """Convert a pandas Series to float32, handling non-string values"""
    if series.dtype == np.object_:
        # Handle non-string values
        return series.apply(lambda x: float(x) if isinstance(x, str) else x)
    else:
        # Handle string values
        return series.astype(np.float32)

class CustomDataset(Dataset):
    """Custom dataset class for loading CSV data"""
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file).apply(convert_to_float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.tensor(self.data.iloc[idx, :-1].values, dtype=torch.float32)
        y = torch.tensor(self.data.iloc[idx, -1], dtype=torch.long)
        return x, y

In [4]:
# Chuẩn bị dữ liệu
dataset = CustomDataset(csv_file)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Khởi tạo mô hình và bộ tối ưu hóa
input_size = num_sensors  # Số lượng cảm biến
model = CNN1D(input_size=input_size, num_classes=num_classes)

ValueError: could not convert string to float: '[2.07,2.07,2.07,2.07,2.06,2.04,2.03,2.02,2.0,2.0]'

In [ ]:
# Huấn luyện mô hình
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs.permute(0, 2, 1))  # Permute để chuyển đổi chiều
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")

print("Training Finished.")

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.